In [8]:
import pandas as pd
import os
import nltk
from sklearn.model_selection import GridSearchCV
import numpy as np
from nltk.corpus import stopwords
from autocorrect import Speller
from nltk.tokenize import word_tokenize
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split
from sklearn.decomposition import PCA
from sklearn.svm import SVC
from sklearn.linear_model import LogisticRegression
from transformers import TFBertTokenizer
from tqdm import tqdm
import spacy
import re
nlp = spacy.load('en_core_web_sm')
import torch.nn as nn
from nltk.stem import WordNetLemmatizer
lemma = WordNetLemmatizer()

import warnings
warnings.filterwarnings('ignore')

In [9]:
data = pd.read_csv("uncleaned_combined_txt.csv")

In [10]:
data

,label,text
0,0,My autistic 3 year old said her first sentence...
1,0,From having the left side of my lips almost co...
2,0,"After dating for just over two years, I propos..."
3,0,One day at a time. Smiling more makes a big di...
4,0,trying my hardest to be happy today. I hope my...
...,...,...
1759,1,death sound appealingthought death thing bring...
1760,1,left relationship almost 5 years depression en...
1761,1,always wanted friendlife never anyone loved ca...
1762,1,deal someone avoids everythinghi close friend ...


In [11]:
# preprocessing funciton created
def preprocessing(df):
    df1 = df.copy()
    df1['white_space_removed'] = 0
    df1['numbers_removed']=0
    df1['url_removed_data'] = 0
    df1['user_removed'] = 0
    df1['emoji_removed'] = 0
    df1['tokenized_data'] = 0
    df1['stopword_removed_data'] = 0
    df1['punct_removed_data'] = 0
    df1['spelling_checked_data'] = 0
    df1['lemma_data'] = 0
    stop_words = set(stopwords.words("english"))

#   lemmatizer = WordNetLemmatizer()
    spell = Speller(lang='en')


  # iterate over each row of dataset and preprocess data
    for i in tqdm(range(df1.shape[0])):

        # white space removel
        df1['white_space_removed'][i] = re.sub("\s+", " ", df1.text[i])
        df1['url_removed_data'][i] = re.sub('(www|http)\S+'," ",df1['white_space_removed'][i])
        df1['user_removed'][i] = re.sub('^@\S+'," ",df1['url_removed_data'][i])
        df1['numbers_removed'][i] = re.sub('\d+','',df1['user_removed'][i])
        emoji_removed = re.sub(r'\W+', ' ', df1['numbers_removed'][i].encode('ascii', 'ignore').decode('utf-8'), flags=re.UNICODE).strip()
        df1['emoji_removed'][i] = emoji_removed

        # lower casing and tokenization
        lower = df1['emoji_removed'][i].lower()
        tokenized_data = word_tokenize(lower)
        df1['tokenized_data'][i] = tokenized_data
        # print(tokenized_data)
        
        


        # remove stopwords
        stopword_removed_data = [x for x in tokenized_data if x not in stop_words]
        df1['stopword_removed_data'][i] = stopword_removed_data 
        # print(stopword_removed_data)
        
        
        # punctuation removal
        punct_removed_data = [x for x in stopword_removed_data if x.isalnum()]
        df1['punct_removed_data'][i] = punct_removed_data
        # print(punct_removed_data)
        

        spelling_checked_data = [spell(x) for x in punct_removed_data]
        df1['spelling_checked_data'][i] = spelling_checked_data
#         print(" ".join(spelling_checked_data))

        lemma_data = [lemma.lemmatize(x) for x in spelling_checked_data]
        df1['lemma_data'][i] = " ".join(lemma_data)

        # spelling checking

    return df1

In [12]:
data.iloc[0]

label                                                    0
text     My autistic 3 year old said her first sentence...
Name: 0, dtype: object

In [14]:
preprocessed_data = preprocessing(data)
preprocessed_data = preprocessed_data[preprocessed_data['lemma_data']!=""]

100%|██████████████████████████████████████████████████████████████████████████████| 1764/1764 [06:42<00:00,  4.38it/s]


In [19]:
df = pd.DataFrame(columns = ['text', 'target'],index = None)
df['text'] = preprocessed_data['lemma_data']
df['target'] = data['label']

In [20]:
df

,text,target
0,autistic year old said first sentence today ne...,0
1,left side lip almost completely torn face dog ...,0
2,dating two year proposed said yes engaged,0
3,one day time smiling make big difference,0
4,trying hardest happy today hope face brings so...,0
...,...,...
1759,death sound appealingthought death thing bring...,1
1760,left relationship almost year depression engul...,1
1761,always wanted friendly never anyone loved care...,1
1762,deal someone avoids everything close friend le...,1


In [21]:
df.to_csv("cleaned_data_all.csv", index = False)
cleaned_df = pd.read_csv("cleaned_data_all.csv")